<a href="https://colab.research.google.com/github/Eucomic/project/blob/main/eskimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Game 클래스와 이동방향
기본적인 게임을 실행하기 위해 방향을 정의하고, 보드에서 방향에 따른 이동과, 그 이동이 성립하는지 여부를 확인하는 함수를 설정하였습니다.

In [10]:
class Game:
    dir_map = {
        'U':  (-1,  0), 'D': (1,  0),
        'R':  (0,  1), 'L': (0, -1),
        'UR': (-1,  1), 'DR': (1,  1),
        'UL': (-1, -1), 'DL': (1, -1),
    }

    def __init__(self):
        self.board = [[0]*5 for _ in range(5)]
        for c in range(5):
            self.board[0][c] = 1
            self.board[4][c] = 2
        self.board[2][2] = 3
        self.bear_cooldown = 0

    def valid_player_move(self, player, sr, sc, dir_str):
        # 1) 입력된 위치의 말이 자신의 말인지를 확인합니다.
        if not (0 <= sr < 5 and 0 <= sc < 5): return None
        if self.board[sr][sc] != player:         return None
        # 2) 입력된 알파벳이 유효한 방향을 입력한 것인지 확인합니다.
        if dir_str not in self.dir_map:          return False
        dr, dc = self.dir_map[dir_str]
        # 3) 해당 방향의 한 칸 앞이 비어있는지 확인합니다. 비어있지 않은 경우, 이동이 불가능하도록 하였습니다.
        nr, nc = sr+dr, sc+dc
        if not (0 <= nr < 5 and 0 <= nc < 5):    return False
        if self.board[nr][nc] != 0:              return False
        # 4) 해당 방향으로 이동 가능할 때 까지 이동합니다.
        r, c = nr, nc
        while True:
            nr, nc = r+dr, c+dc
            if not (0 <= nr < 5 and 0 <= nc < 5): break
            if self.board[nr][nc] != 0:          break
            r, c = nr, nc
        return (r, c)

    def valid_bear_move(self, sr, sc, dir_str):
        # 1) 쿨다운 값이 0인지 확인합니다.
        if self.bear_cooldown > 0:    return None
        # 2) 입력된 위치에 북극곰 말이 있는지 확인합니다.
        if not (0 <= sr < 5 and 0 <= sc < 5): return None
        if self.board[sr][sc] != 3:          return None
        # 3) 입력된 알파벳이 유효한 방향을 입력한 것인지 확인합니다.
        if dir_str not in self.dir_map:      return False
        # 4) 한 칸 앞(도착지점)이 비어있는지 확인합니다.
        dr, dc = self.dir_map[dir_str]
        tr, tc = sr+dr, sc+dc
        if not (0 <= tr < 5 and 0 <= tc < 5): return False
        if self.board[tr][tc] != 0:          return False
        # 5) 도착지점으로 말을 이동시킵니다.
        return (tr, tc)

    def is_bear_trapped(self):
        # 1) 북극곰 말의 위치를 파악합니다.
        for r in range(5):
            for c in range(5):
                if self.board[r][c] == 3:
                    bear_r, bear_c = r, c
        # 2) 북극곰의 상/하/좌/우에 위치한 칸이 빈칸인지를 확인합니다.
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = bear_r+dr, bear_c+dc
            if 0 <= nr < 5 and 0 <= nc < 5 and self.board[nr][nc] == 0:
                return False
        return True


# 2. 보드 제작
행과 열의 번호를 포함한 전체적인 보드의 구성을 제작하였습니다. 처음에 격자가 존재하지 않는 상태(ver1)로 진행하려다가, 조금 더 직관적인 구성을 위해 행과 열의 번호를 포함하고 격자가 들어가도록 형태를 변경했습니다.(ver2)


In [11]:
# ver 1

# def print_board(bd):
#     # 0: . , 1: X (P1), 2: O (P2), 3: B (Bear)
#     symbol = {0:'.', 1:'X', 2:'O', 3:'B'}
#     for row in bd:
#         print(' '.join(symbol[v] for v in row))
#     print()  # 한 줄 띄우기

# ver 2

def print_board(bd):
    symbol = {0:'.', 1:'1', 2:'2', 3:'B'}
    size = len(bd)
    header = '   ' + '   '.join(str(i+1) for i in range(size))
    print(header)
    print('  +' + '---+' * size)
    for i, row in enumerate(bd):
        print(f"{i+1} | " + ' | '.join(symbol[v] for v in row) + ' |')
        print('  +' + '---+' * size)


# 3. 게임 실행
게임을 실행하기 위한 함수입니다. 최종적인 북극곰을 가두는 형태가 완성될 때 까지, 반복하여 번갈아가며 두 플레이어의 행동을 입력받습니다.

In [12]:
def play_game():
    game = Game()
    current = 1
    while True:
        print_board(game.board)
        # 이동을 입력받습니다.
        raw = input(f"[P{current}] 이동 (예: '2 3 U' or 'bear 3 3 DR'): ")

        # 입력 받은 텍스트를 띄어쓰기를 기준으로 분리합니다.
        cmd = raw.strip().split()

        # 입력이 없는 경우
        if not cmd:
            print("⚠️ 입력이 없습니다.")
            continue

        # 북극곰 이동인 경우
        if cmd[0].lower() == 'bear':
            # 입력 값이 부족한 경우(행/열/방향 중 하나 이상 누락)
            if len(cmd) < 4:
                print("⚠️ 포맷: bear row col DIR")
                continue
            sr, sc = int(cmd[1])-1, int(cmd[2])-1
            dir_str = cmd[3].upper()
            res = game.valid_bear_move(sr, sc, dir_str)
            # Cell 1 : 북극곰 이동 - 1/2번의 오류 상황
            if res is None:
                print("❌ 잘못된 곰 선택 또는 쿨다운 중")
                continue
            # Cell 1 : 북극곰 이동 - 3/4의 오류 상황
            if res is False:
                print("❌ 곰은 그 방향으로 이동할 수 없습니다.")
                continue
            tr, tc = res
            # 정상 이동 후, 이동 전 칸은 빈칸, 이동 후 칸은 북극곰으로 설정, 쿨다운 초기화
            game.board[sr][sc], game.board[tr][tc] = 0, 3
            game.bear_cooldown = 4

        else:
            # 입력 값이 부족한 경우(행/열/방향 중 하나 이상 누락)
            if len(cmd) < 3:
                print("⚠️ 포맷: row col DIR")
                continue
            sr, sc = int(cmd[0])-1, int(cmd[1])-1
            dir_str = cmd[2].upper()
            res = game.valid_player_move(current, sr, sc, dir_str)
            # Cell 1 : 플레이어 말 이동 - 1번의 오류 상황
            if res is None:
                print("❌ 내 말이 아닙니다.")
                continue
            # Cell 1 : 플레이어 말 이동 - 2/3번의 오류 상황
            if res is False:
                print("❌ 그 방향으로 이동할 수 없습니다.")
                continue
            tr, tc = res
            # 정상 이동 후, 이동 전 칸은 빈칸, 이동 후 칸은 현재 플레이어의 말로 설정
            game.board[sr][sc], game.board[tr][tc] = 0, current

        # 곰 쿨다운 감소
        game.bear_cooldown = max(0, game.bear_cooldown - 1)
        # 게임 종료 조건 확인
        if game.is_bear_trapped():
            print_board(game.board)
            print(f"P{current} 승리!")
            break
        current = 3 - current

# 한 번에 실행
if __name__ == "__main__":
    play_game()


   1   2   3   4   5
  +---+---+---+---+---+
1 | 1 | 1 | 1 | 1 | 1 |
  +---+---+---+---+---+
2 | . | . | . | . | . |
  +---+---+---+---+---+
3 | . | . | B | . | . |
  +---+---+---+---+---+
4 | . | . | . | . | . |
  +---+---+---+---+---+
5 | 2 | 2 | 2 | 2 | 2 |
  +---+---+---+---+---+
[P1] 이동 (예: '2 3 U' or 'bear 3 3 DR'): 1 3 D
   1   2   3   4   5
  +---+---+---+---+---+
1 | 1 | 1 | . | 1 | 1 |
  +---+---+---+---+---+
2 | . | . | 1 | . | . |
  +---+---+---+---+---+
3 | . | . | B | . | . |
  +---+---+---+---+---+
4 | . | . | . | . | . |
  +---+---+---+---+---+
5 | 2 | 2 | 2 | 2 | 2 |
  +---+---+---+---+---+


KeyboardInterrupt: Interrupted by user